In [ ]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com
     |████████████████████████████████| 11.1 MB 6.2 MB/s 


In [ ]:
import numpy as np
from gurobipy import *
import math
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

def supp(c):
    """LP oracle that solves max c^T w subject to w in S
       where S is linear constraints in the shortest path problem on 5 x 5 grid """

    edges = tuplelist(((1,2), (2,3), (1,4), (2,5), (3,6), (4,5), (5,6),\
                      (4,7), (5,8), (6,9), (7,8), (8,9)))
    origin = 1
    destination = 9

    num_nodes = 9

    cost = dict(zip(edges, c))

    m = Model()
    w = m.addVars(edges, obj=cost)
    m.ModelSense = GRB.MAXIMIZE
    m.Params.LogToConsole = 0 # Hide logs

    for i in range(1, num_nodes+1):
        m.addConstr( sum(w[i,j] for i,j in edges.select(i, '*')) - sum(w[j,i] for j,i in edges.select('*',i)) == 
                     (1 if i==origin else -1 if i==destination else 0 ),'node%s_' % i )

    m.optimize() 
    z = m.objval # objective value
    w = np.array([i.x for i in m.getVars()]) # argmax weight vector
    m.reset()

    return z, w

def w_star(c):
    """returns argmin c^T w subject to w in S"""
    return supp(-c)[1]


def spo_loss(c_true, c_pred):
    return np.dot(c_true, w_star(c_pred)) - np.dot(c_true, w_star(c_true))


# Set Fixed Values

d = 12
p = 3

In [ ]:
class Sim():
    def __init__(self, n_train, deg, eps):
        self.n_train = n_train
        self.deg = deg
        self.eps = eps
        self.init = np.random.normal(0, 1, (d,p))
        self.B = np.zeros((d,p))
        self.B_star = np.random.binomial(size = (d,p), n = 1, p = 0.5)

        self.X_train = np.zeros((self.n_train,p))      
        for i in range(self.n_train):
            self.X_train[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_train = np.zeros((self.n_train,d))
        for i in range(self.n_train):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_train[i,j] = np.power((self.B_star @ self.X_train[i])[j] / np.sqrt(p) + 3, self.deg) * noise
        
        self.n_test = 1000

        self.X_test = np.zeros((self.n_test,p))
        for i in range(self.n_test):
            self.X_test[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))
        self.c_test = np.zeros((self.n_test,d))
        for i in range(self.n_test):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_test[i,j] = np.power((self.B_star @ self.X_test[i])[j] / np.sqrt(p) + 3, self.deg) * noise

        self.dca_norm_spo_loss = 0
        self.sp_norm_spo_loss = 0 

    def subgrad_g(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(alpha * B @ X[i] - c[i]), X[i]) for i in range(n)], axis=0)
        return s / n

    def subgrad_h(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(B @ X[i]), X[i]) for i in range(n)], axis=0)
        return s / n 

    def G(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] for i in range(n)])
        return s / n

    def H(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([alpha * supp(- B @ X[i])[0] for i in range(n)])
        return s / n

    def DC(self, B, X, c, alpha):
        return self.G(B, X, c, alpha) - self.H(B, X, c, alpha)
    
    def dc_subgrd_solver(self, B_init, C, X, c, alpha, gamma, T, tol):
        # Minimize G(B) - <B,C>
        B = B_init
        
        curr_min = math.inf
        for k in range(T):
            B_new = B - gamma * (self.subgrad_g(B, X, c, alpha) - C)

            obj_val = self.G(B_new, X, c, alpha) - np.sum(np.multiply(B_new, C))
            # print("obj_val",obj_val, "curr min", curr_min)
            if obj_val <= curr_min:
                gap = curr_min - obj_val
                curr_min = obj_val
                B_best = B_new
                print("iteration", k, gap)
                if gap < tol:
                    break

            B = B_new

        if k == T - 1:
            B_new = B_best

        return B_new

    def fit_dca(self, alpha, gamma):
        self.B = self.init

        T = 1000 # number of iterations
        tol = 1e-5 # tolerance

        for k in range(T):
            C = self.subgrad_h(self.B, self.X_train, self.c_train, alpha)
            B_new = self.dc_subgrd_solver(np.ones((d,p)), C, self.X_train, self.c_train, alpha, gamma, T, tol)
    
            dc_gap = np.abs(self.DC(B_new, self.X_train, self.c_train, alpha) \
                      - self.DC(self.B, self.X_train, self.c_train, alpha))
            
            if dc_gap < tol:
                break
            print("Epoch:", k, "DC Gap:", dc_gap, "DC Objective Value:", self.DC(B_new, self.X_train, self.c_train, alpha))
            
            self.B = B_new


        print("DCA Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i]) \
                                                          for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.dca_norm_spo_loss = numerator / denominator


    def subgrad_sp(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(alpha * (w_star(c[i]) - w_star(alpha * B @ X[i] - c[i])), X[i]) for i in range(n)], axis=0)
        return s / n

    def spo_plus(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] + alpha * np.dot(B @ X[i], w_star(c[i])) for i in range(n)])
        return s / n
    
    def spo_subgrd_solver(self, B_init, X, c, alpha, gamma, T, tol):
        B = B_init
        curr_min = math.inf
        for k in range(T):
            B_new = B - (gamma/np.sqrt(k+1)) * self.subgrad_sp(B, X, c, alpha) 
            
            obj_val = self.spo_plus(B_new, X, c, alpha)
            # print("obj_val", obj_val, "curr min", curr_min)
            if obj_val <= curr_min:
                gap = curr_min - obj_val
                curr_min = obj_val
                B_best = B_new
                print("iteration", k, gap, "SPO+ Objective Value:", obj_val)
                if gap < tol:
                    break
                    
            B = B_new
            
        if k == T - 1:
            B_new = B_best

        return B_new

    def fit_sp(self, alpha, gamma):
        self.B = self.init
    
        T = 1000 # number of iterations
        tol = 1e-5 # tolerance

        B_new = self.spo_subgrd_solver(self.B, self.X_train, self.c_train, alpha, gamma, T, tol)
            
        print("SPO+ Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i])\
                                                           for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.sp_norm_spo_loss = numerator / denominator

In [ ]:
deg = 1
eps = 0

num_trials = 5
DCA_loss = []
SP_loss = []

for trials in range(num_trials):
    sim = Sim(n_train=100, deg=deg, eps=eps)
    sim.fit_dca(alpha = 2, gamma = 0.5)
    print("DCA Norm SPO Loss:", sim.dca_norm_spo_loss)
    DCA_loss.append(sim.dca_norm_spo_loss)
    sim.fit_sp(alpha = 2, gamma = 0.5)
    print("SPO+ Norm SPO Loss:", sim.sp_norm_spo_loss)
    SP_loss.append(sim.sp_norm_spo_loss)



# DATAFRAMES WITH TRIAL COLUMN ASSIGNED
df1 = pd.DataFrame(np.vstack((DCA_loss, SP_loss)).T, columns=["DCA", "SPO+"]).assign(deg=1)

cdf = pd.concat([df1])                                # CONCATENATE
mdf = pd.melt(cdf, id_vars=['deg'], var_name=['Loss'])      # MELT

ax = sns.boxplot(x="deg", y="value", hue="Loss", data=mdf)  # RUN PLOT   
plt.show()

plt.clf()
plt.close()

obj_val 12.634841595458184 curr min inf
iteration 0 inf
obj_val 12.147065681068563 curr min 12.634841595458184
iteration 1 0.487775914389621
obj_val 12.011511688949131 curr min 12.147065681068563
iteration 2 0.13555399211943175
obj_val 12.0630089565351 curr min 12.011511688949131
obj_val 12.011601804979403 curr min 12.011511688949131
obj_val 11.992929640061345 curr min 12.011511688949131
iteration 5 0.018582048887786584
obj_val 12.003859693808081 curr min 11.992929640061345
obj_val 11.987054272920151 curr min 11.992929640061345
iteration 7 0.0058753671411935215
obj_val 12.003229435786142 curr min 11.987054272920151
obj_val 11.999154377225958 curr min 11.987054272920151
obj_val 11.988930105559891 curr min 11.987054272920151
obj_val 11.996833705755929 curr min 11.987054272920151
obj_val 11.982919024544616 curr min 11.987054272920151
iteration 12 0.004135248375535028
obj_val 11.974979308027692 curr min 11.982919024544616
iteration 13 0.007939716516924378
obj_val 11.966973439308637 curr mi

KeyboardInterrupt: 

In [ ]:
class Sim1():
    # Trick 1: Add rho||x||^2 /2 to the objectives 
    def __init__(self, n_train, deg, eps):
        self.n_train = n_train
        self.deg = deg
        self.eps = eps
        self.init = np.random.normal(0, 1, (d,p))
        self.B = np.zeros((d,p))
        self.B_star = np.random.binomial(size = (d,p), n = 1, p = 0.5)

        self.X_train = np.zeros((self.n_train,p))      
        for i in range(self.n_train):
            self.X_train[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_train = np.zeros((self.n_train,d))
        for i in range(self.n_train):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_train[i,j] = np.power((self.B_star @ self.X_train[i])[j] / np.sqrt(p) + 3, self.deg) * noise
        
        self.n_test = 1000

        self.X_test = np.zeros((self.n_test,p))
        for i in range(self.n_test):
            self.X_test[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_test = np.zeros((self.n_test,d))
        for i in range(self.n_test):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_test[i,j] = np.power((self.B_star @ self.X_test[i])[j] / np.sqrt(p) + 3, self.deg) * noise

        self.dca_norm_spo_loss = 0
        self.sp_norm_spo_loss = 0 

    def subgrad_g(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(alpha * B @ X[i] - c[i]), X[i]) for i in range(n)], axis=0)
        return s / n + rho * B

    def subgrad_h(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(B @ X[i]), X[i]) for i in range(n)], axis=0)
        return s / n + rho * B

    def G(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] for i in range(n)])
        return s / n + (rho / 2) * np.sum(np.multiply(B,B))

    def H(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([alpha * supp(- B @ X[i])[0] for i in range(n)])
        return s / n + (rho / 2) * np.sum(np.multiply(B,B))

    def DC(self, B, X, c, alpha, rho):
        return self.G(B, X, c, alpha, rho) - self.H(B, X, c, alpha, rho)
    
    def dc_subgrd_solver(self, B_init, C, X, c, alpha, gamma, rho, T, tol):
        # Minimize G(B) - <B,C>
        B = B_init
        
        curr_min = math.inf
        for k in range(T):
            B_new = B - (gamma/np.sqrt(k+1)) * (self.subgrad_g(B, X, c, alpha, rho) - C)

            obj_val = self.G(B_new, X, c, alpha, rho) - np.sum(np.multiply(B_new, C))
#             print("obj_val",obj_val, "curr min", curr_min)
            if obj_val <= curr_min:
                gap = curr_min - obj_val
                curr_min = obj_val
                B_best = B_new
                print("iteration", k, gap)
                if gap < tol:
                    break

            B = B_new

        if k == T - 1:
            B_new = B_best

        return B_new

    def fit_dca(self, alpha, gamma, rho = 2):
        self.B = self.init

        T = 1000 # number of iterations
        tol = 1e-5 # tolerance

        for k in range(T):
            C = self.subgrad_h(self.B, self.X_train, self.c_train, alpha, rho)
            B_new = self.dc_subgrd_solver(np.ones((d,p)), C, self.X_train, self.c_train, alpha, gamma, rho, T, tol)
    
            dc_gap = np.abs(self.DC(B_new, self.X_train, self.c_train, alpha, rho) \
                      - self.DC(self.B, self.X_train, self.c_train, alpha, rho))
            
            if dc_gap < tol:
                break
            print("Epoch:", k, "DC Gap:", dc_gap, "DC Objective Value:", self.DC(B_new, self.X_train, self.c_train, alpha, rho))
            
            self.B = B_new


        print("DCA Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i]) \
                                                          for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.dca_norm_spo_loss = numerator / denominator


    def subgrad_sp(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(alpha * (w_star(c[i]) - w_star(alpha * B @ X[i] - c[i])), X[i]) for i in range(n)], axis=0)
        return s / n

    def spo_plus(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] + alpha * np.dot(B @ X[i], w_star(c[i])) for i in range(n)])
        return s / n

    def spo_subgrd_solver(self, B_init, X, c, alpha, gamma, T, tol):
        B = B_init
        curr_min = math.inf
        for k in range(T):
            B_new = B - (gamma/np.sqrt(k+1)) * self.subgrad_sp(B, X, c, alpha) 
            
            obj_val = self.spo_plus(B_new, X, c, alpha)
            # print("obj_val", obj_val, "curr min", curr_min)
            if obj_val <= curr_min:
                gap = curr_min - obj_val
                curr_min = obj_val
                B_best = B_new
                print("iteration", k, gap, "SPO+ Objective Value:", obj_val)
                if gap < tol:
                    break
                    
            B = B_new
            
        if k == T - 1:
            B_new = B_best

        return B_new

    def fit_sp(self, alpha, gamma):
        self.B = self.init
    
        T = 1000 # number of iterations
        tol = 1e-5 # tolerance

        B_new = self.spo_subgrd_solver(self.B, self.X_train, self.c_train, alpha, gamma, T, tol)
            
        print("SPO+ Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i])\
                                                           for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.sp_norm_spo_loss = numerator / denominator

In [ ]:
deg = 4
eps = 0

num_trials = 5
DCA_loss = []
SP_loss = []

for trials in range(num_trials):
    sim = Sim1(n_train=100, deg=deg, eps=eps)
    sim.fit_dca(alpha = 2, gamma = 0.5)
    print("DCA Norm SPO Loss:", sim.dca_norm_spo_loss)
    DCA_loss.append(sim.dca_norm_spo_loss)
    sim.fit_sp(alpha = 2, gamma = 0.5)
    print("SPO+ Norm SPO Loss:", sim.sp_norm_spo_loss)
    SP_loss.append(sim.sp_norm_spo_loss)



# DATAFRAMES WITH TRIAL COLUMN ASSIGNED
df1 = pd.DataFrame(np.vstack((DCA_loss, SP_loss)).T, columns=["DCA", "SPO+"]).assign(deg=1)

cdf = pd.concat([df1])                                # CONCATENATE
mdf = pd.melt(cdf, id_vars=['deg'], var_name=['Loss'])      # MELT

ax = sns.boxplot(x="deg", y="value", hue="Loss", data=mdf)  # RUN PLOT   
plt.show()

plt.clf()
plt.close()

Academic license - for non-commercial use only - expires 2022-09-01
Using license file C:\Users\Hong-Seok Choe\gurobi.lic
iteration 0 inf
iteration 1 0.05520368784618768
iteration 2 0.0011976865941960568
iteration 3 0.00019535668911885296
iteration 4 4.522033270859538e-05
iteration 5 1.2930682032674667e-05
iteration 6 4.271811690159666e-06
Epoch: 0 DC Gap: 3.9364016627644105 DC Objective Value: 570.5315005771491
iteration 0 inf
iteration 1 0.05154829027173946
iteration 2 0.0010445817824802361
iteration 3 0.00017038350404163793
iteration 4 3.943964611607953e-05
iteration 5 1.1277704061285476e-05
iteration 6 3.7257300391502213e-06
Epoch: 1 DC Gap: 5.726629838292979 DC Objective Value: 564.8048707388562
iteration 0 inf
iteration 1 0.059934078086257614
iteration 2 0.000915430655709315
iteration 3 0.0001493174448796708
iteration 4 3.4563364806672325e-05
iteration 5 9.883339316729689e-06
Epoch: 2 DC Gap: 6.755435711820951 DC Objective Value: 558.0494350270352
iteration 0 inf
iteration 1 0.07

In [ ]:
class SimProx():
    def __init__(self, n_train, deg, eps):
        self.n_train = n_train
        self.deg = deg
        self.eps = eps
        self.init = np.random.normal(0, 1, (d,p))
        self.B = np.zeros((d,p))
        self.B_star = np.random.binomial(size = (d,p), n = 1, p = 0.5)

        self.X_train = np.zeros((self.n_train,p))      
        for i in range(self.n_train):
            self.X_train[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_train = np.zeros((self.n_train,d))
        for i in range(self.n_train):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_train[i,j] = np.power((self.B_star @ self.X_train[i])[j] / np.sqrt(p) + 3, self.deg) * noise
        
        self.n_test = 1000

        self.X_test = np.zeros((self.n_test,p))
        for i in range(self.n_test):
            self.X_test[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_test = np.zeros((self.n_test,d))
        for i in range(self.n_test):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_test[i,j] = np.power((self.B_star @ self.X_test[i])[j] / np.sqrt(p) + 3, self.deg) * noise

        self.dca_norm_spo_loss = 0
        self.sp_norm_spo_loss = 0 

    def subgrad_g(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(alpha * B @ X[i] - c[i]), X[i]) for i in range(n)], axis=0)
        return s / n

    def subgrad_h(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(B @ X[i]), X[i]) for i in range(n)], axis=0)
        return s / n

    def G(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] for i in range(n)])
        return s / n

    def H(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([alpha * supp(- B @ X[i])[0] for i in range(n)])
        return s / n

    def DC(self, B, X, c, alpha):
        return self.G(B, X, c, alpha) - self.H(B, X, c, alpha)

    def dc_subgrd_solver(self, B_init, C, X, c, alpha, gamma, s, T, tol):
        # Minimize G(B) + (1/2s)||B - C||^2
        B = B_init
        
        curr_min = math.inf
        for k in range(T):
            B_new = B - (gamma/np.sqrt(k+1)) * (self.subgrad_g(B, X, c, alpha) + (1/s) * (B - C))

            obj_val = self.G(B_new, X, c, alpha) + (1/(2*s)) * np.sum(np.multiply(B_new - C, B_new - C))
            # print("obj_val",obj_val, "curr min", curr_min)
            if obj_val < curr_min:
                gap = curr_min - obj_val
                curr_min = obj_val
                B_best = B_new
                print("iteration", k, gap)
                if gap < tol:
                    break

            B = B_new

        if k == T - 1:
            B_new = B_best

        return B_new   

    def fit_dca(self, alpha, gamma, s = 1): # s = step size in proximal iteration
        self.B = self.init

        T = 5000 # number of iterations
        tol = 1e-5 # tolerance

        for k in range(T):
            C = self.subgrad_h(self.B, self.X_train, self.c_train, alpha)
            B_new = self.dc_subgrd_solver(np.ones((d,p)), self.B + s * C, self.X_train, self.c_train, alpha, gamma, s, T, tol)

            dc_gap = np.abs(self.DC(B_new, self.X_train, self.c_train, alpha) \
                      - self.DC(self.B, self.X_train, self.c_train, alpha))
            if dc_gap < tol:
                break
            print("Epoch:", k, "DC Gap:", dc_gap, "DC Objective Value:", self.DC(B_new, self.X_train, self.c_train, alpha))
            
            self.B = B_new


        print("DCA Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i]) \
                                                          for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.dca_norm_spo_loss = numerator / denominator


    def subgrad_sp(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(alpha * (w_star(c[i]) - w_star(alpha * B @ X[i] - c[i])), X[i]) for i in range(n)], axis=0)
        return s / n

    def spo_plus(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] + alpha * np.dot(B @ X[i], w_star(c[i])) for i in range(n)])
        return s / n

    def spo_subgrd_solver(self, B_init, X, c, alpha, gamma, T, tol):
        B = B_init
        curr_min = math.inf
        for k in range(T):
            B_new = B - (gamma/np.sqrt(k+1)) * self.subgrad_sp(B, X, c, alpha) 
            
            obj_val = self.spo_plus(B_new, X, c, alpha)
            # print("obj_val", obj_val, "curr min", curr_min)
            if obj_val <= curr_min:
                gap = curr_min - obj_val
                curr_min = obj_val
                B_best = B_new
                print("iteration", k, gap, "SPO+ Objective Value:", obj_val)
                if gap < tol:
                    break
                    
            B = B_new
            
        if k == T - 1:
            B_new = B_best

        return B_new

    def fit_sp(self, alpha, gamma):
        self.B = self.init
    
        T = 100000 # number of iterations
        tol = 1e-5 # tolerance

        B_new = self.spo_subgrd_solver(self.B, self.X_train, self.c_train, alpha, gamma, T, tol)
            
        print("SPO+ Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i])\
                                                           for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.sp_norm_spo_loss = numerator / denominator

In [ ]:
deg = 4
eps = 0

num_trials = 1
DCA_loss = []
SP_loss = []

for trials in range(num_trials):
    sim = SimProx(n_train=100, deg=deg, eps=eps)
    sim.fit_dca(alpha = 2, gamma = 0.5)
    print("DCA Norm SPO Loss:", sim.dca_norm_spo_loss)
    DCA_loss.append(sim.dca_norm_spo_loss)
    sim.fit_sp(alpha = 2, gamma = 0.5)
    print("SPO+ Norm SPO Loss:", sim.sp_norm_spo_loss)
    SP_loss.append(sim.sp_norm_spo_loss)



# DATAFRAMES WITH TRIAL COLUMN ASSIGNED
df1 = pd.DataFrame(np.vstack((DCA_loss, SP_loss)).T, columns=["DCA", "SPO+"]).assign(deg=1)

cdf = pd.concat([df1])                                # CONCATENATE
mdf = pd.melt(cdf, id_vars=['deg'], var_name=['Loss'])      # MELT

ax = sns.boxplot(x="deg", y="value", hue="Loss", data=mdf)  # RUN PLOT   
plt.show()

plt.clf()
plt.close()

Streaming output truncated to the last 5000 lines.
iteration 43012 7.9703266294473e-05 SPO+ Objective Value: 303.87675169498766
iteration 43013 6.962151422840179e-05 SPO+ Objective Value: 303.87668207347343
iteration 43014 8.960681799408121e-05 SPO+ Objective Value: 303.87659246665544
iteration 43015 7.975257847192552e-05 SPO+ Objective Value: 303.87651271407697
iteration 43016 7.502241157908429e-05 SPO+ Objective Value: 303.8764376916654
iteration 43017 8.322878647959442e-05 SPO+ Objective Value: 303.8763544628789
iteration 43018 7.16691594675467e-05 SPO+ Objective Value: 303.87628279371944
iteration 43019 8.960161051163595e-05 SPO+ Objective Value: 303.87619319210893
iteration 43020 7.798310787165974e-05 SPO+ Objective Value: 303.87611520900106
iteration 43021 7.791919142619008e-05 SPO+ Objective Value: 303.87603728980963
iteration 43022 8.003595524996854e-05 SPO+ Objective Value: 303.8759572538544
iteration 43023 7.371668601763304e-05 SPO+ Objective Value: 303.87588353716836
iterati

In [ ]:
print(DCA_loss)
print(SP_loss)

In [ ]:
import math

class SimProx():
    def __init__(self, n_train, deg, eps):
        self.n_train = n_train
        self.deg = deg
        self.eps = eps
        self.init = np.random.normal(0, 1, (d,p))
        self.B = np.zeros((d,p))
        self.B_star = np.random.binomial(size = (d,p), n = 1, p = 0.5)

        self.X_train = np.zeros((self.n_train,p))      
        for i in range(self.n_train):
            self.X_train[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_train = np.zeros((self.n_train,d))
        for i in range(self.n_train):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_train[i,j] = np.power((self.B_star @ self.X_train[i])[j] / np.sqrt(p) + 3, self.deg) * noise
        
        self.n_test = 1000

        self.X_test = np.zeros((self.n_test,p))
        for i in range(self.n_test):
            self.X_test[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_test = np.zeros((self.n_test,d))
        for i in range(self.n_test):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_test[i,j] = np.power((self.B_star @ self.X_test[i])[j] / np.sqrt(p) + 3, self.deg) * noise

        self.dca_norm_spo_loss = 0
        self.sp_norm_spo_loss = 0 

    def subgrad_g(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(alpha * B @ X[i] - c[i]), X[i]) for i in range(n)], axis=0)
        return s / n

    def subgrad_h(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(B @ X[i]), X[i]) for i in range(n)], axis=0)
        return s / n

    def G(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] for i in range(n)])
        return s / n

    def H(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([alpha * supp(- B @ X[i])[0] for i in range(n)])
        return s / n

    def DC(self, B, X, c, alpha):
        return self.G(B, X, c, alpha) - self.H(B, X, c, alpha)

    def argmin_obj(self, B, C, X, c, alpha, s):
        return self.G(B, X, c, alpha) - (1/(2*s)) * np.sum(np.multiply(B - C, B - C))

    def fit_dca(self, alpha, gamma, s=1): # p = step size in proximal iteration
        self.B = self.init

        T = 1000 # number of iterations
        tol = 1e-5 # tolerance

        for k in range(T):
            C = self.subgrad_h(self.B, self.X_train, self.c_train, alpha)

            B_prime = self.B
            curr_min = math.inf
            for l in range(T):
                B_prime_new = B_prime - (gamma/np.sqrt(l+1)) * (self.subgrad_g(B_prime, self.X_train, self.c_train, alpha) + (1/p) * (B_prime - self.B - s * C))

                obj_val = self.argmin_obj(B_prime_new, self.B + s * C, self.X_train, self.c_train, alpha, s)
                print(obj_val, curr_min)
                if obj_val < curr_min:
                    gap = np.abs(curr_min - obj_val)
                    curr_min = obj_val
                    B_best = B_prime_new
                    print("iteration", k, l, gap)
                    if gap < tol:
                        B_new = B_prime_new
                        break
                        
#             for l in range(T):
#                 B_prime_new = B_prime - (gamma/np.sqrt(l+1)) * (self.subgrad_g(B_prime, self.X_train, self.c_train, alpha) + (1/p) * (B_prime - self.B - p * C))

#                 obj_gap = np.abs(self.argmin_obj(B_prime_new, self.B + p * C, self.X_train, self.c_train, alpha, p) \
#                                  - self.argmin_obj(B_prime, self.B + p * C, self.X_train, self.c_train, alpha, p))
#                 print("iteration", k, l, obj_gap)
#                 if obj_gap < tol:
#                     B_new = B_prime_new
#                     break
                B_prime = B_prime_new
    
            if l == T-1:
                B_new = B_best
            
            dc_gap = np.abs(self.DC(B_new, self.X_train, self.c_train, alpha) \
                      - self.DC(self.B, self.X_train, self.c_train, alpha))
            if dc_gap < tol:
                break
            print("DC Gap:", dc_gap)
            
            self.B = B_new


        print("DCA Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i]) \
                                                          for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.dca_norm_spo_loss = numerator / denominator


    def subgrad_sp(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(alpha * (w_star(c[i]) - w_star(alpha * B @ X[i] - c[i])), X[i]) for i in range(n)], axis=0)
        return s / n

    def spo_plus(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] + alpha * np.dot(B @ X[i], w_star(c[i])) for i in range(n)])
        return s / n

    def fit_sp(self, alpha, gamma):
        self.B = self.init
    
        T = 1000 # number of iterations
        tol = 1e-5 # tolerance

#         for k in range(T):
#             B_new = self.B - (gamma/np.sqrt(k+1)) * self.subgrad_sp(self.B, self.X_train, self.c_train, alpha) 
            
            
#             obj_gap = np.abs(self.spo_plus(B_new, self.X_train, self.c_train, alpha) \
#                              - self.spo_plus(self.B, self.X_train, self.c_train, alpha))
#             print("iteration", k, obj_gap)
#             if obj_gap < tol:
#                 break
        curr_min = math.inf
        for k in range(T):
            B_new = self.B - (gamma/np.sqrt(k+1)) * self.subgrad_sp(self.B, self.X_train, self.c_train, alpha) 
            
            obj_val = self.spo_plus(B_new, self.X_train, self.c_train, alpha)
            if obj_val < curr_min:
                gap = np.abs(curr_min - obj_val)
                curr_min = obj_val
                B_best = B_new
                print("iteration", k, gap)
                if gap < tol:
                    break
                    
            self.B = B_new
            
        if k == T-1:
            B_new = B_best
        print("SPO+ Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i])\
                                                           for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.sp_norm_spo_loss = numerator / denominator

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

deg = 1
eps = 0

num_trials = 5
DCA_loss = []
SP_loss = []

for trials in range(num_trials):
    sim = SimProx(n_train=100, deg=deg, eps=eps)
    sim.fit_dca(alpha = 2, gamma = 0.5)
    print("DCA Norm SPO Loss:", sim.dca_norm_spo_loss)
    DCA_loss.append(sim.dca_norm_spo_loss)
    sim.fit_sp(alpha = 2, gamma = 0.5)
    print("SPO+ Norm SPO Loss:", sim.sp_norm_spo_loss)
    SP_loss.append(sim.sp_norm_spo_loss)



# DATAFRAMES WITH TRIAL COLUMN ASSIGNED
df1 = pd.DataFrame(np.vstack((DCA_loss, SP_loss)).T, columns=["DCA", "SPO+"]).assign(deg=1)

cdf = pd.concat([df1])                                # CONCATENATE
mdf = pd.melt(cdf, id_vars=['deg'], var_name=['Loss'])      # MELT

ax = sns.boxplot(x="deg", y="value", hue="Loss", data=mdf)  # RUN PLOT   
plt.show()

plt.clf()
plt.close()

14.53734617018527 inf
iteration 0 0 inf
11.218386344063722 14.53734617018527
iteration 0 1 3.3189598261215476
8.651235997599697 11.218386344063722
iteration 0 2 2.5671503464640253
6.505020847507822 8.651235997599697
iteration 0 3 2.1462151500918747
4.6356912137459485 6.505020847507822
iteration 0 4 1.8693296337618737
3.1118624498351792 4.6356912137459485
iteration 0 5 1.5238287639107693
1.7708295486220695 3.1118624498351792
iteration 0 6 1.3410329012131097
0.7343659602238102 1.7708295486220695
iteration 0 7 1.0364635883982594
-0.3097699773529907 0.7343659602238102
iteration 0 8 1.044135937576801
-1.1673622758316782 -0.3097699773529907
iteration 0 9 0.8575922984786875
-1.9710652342515687 -1.1673622758316782
iteration 0 10 0.8037029584198905
-2.538380762584035 -1.9710652342515687
iteration 0 11 0.5673155283324665
-3.2955147723247755 -2.538380762584035
iteration 0 12 0.7571340097407404
-3.8921703990627545 -3.2955147723247755
iteration 0 13 0.596655626737979
-4.36836883213638 -3.8921703990

-16.950228702125255 -16.946913581258364
iteration 0 122 0.003315120866890453
-16.999624979107544 -16.950228702125255
iteration 0 123 0.04939627698228932
-16.896920664980996 -16.999624979107544
-16.97972470000261 -16.999624979107544
-16.987822792228933 -16.999624979107544
-17.057636941523658 -16.999624979107544
iteration 0 127 0.058011962416113505
-16.924279083224583 -17.057636941523658
-17.02636349194544 -17.057636941523658
-17.045824203708747 -17.057636941523658
-17.098045064940255 -17.057636941523658
iteration 0 131 0.040408123416597164
-16.99795850761777 -17.098045064940255
-17.008915117078985 -17.098045064940255
-17.048357874263672 -17.098045064940255
-17.100564862423163 -17.098045064940255
iteration 0 135 0.0025197974829076486
-17.031205783798022 -17.100564862423163
-17.097563627222073 -17.100564862423163
-17.054442784820555 -17.100564862423163
-17.145673520090995 -17.100564862423163
iteration 0 139 0.04510865766783212
-17.0600039750639 -17.145673520090995
-17.138446347114964 -17.

-17.68476547155342 -17.74189221257751
-17.717928972079456 -17.74189221257751
-17.704495730049175 -17.74189221257751
-17.760869506839093 -17.74189221257751
iteration 0 301 0.01897729426158179
-17.690874342212783 -17.760869506839093
-17.73932777729908 -17.760869506839093
-17.733737682745527 -17.760869506839093
-17.767403081811782 -17.760869506839093
iteration 0 305 0.006533574972689138
-17.72196093247243 -17.767403081811782
-17.763363024255128 -17.767403081811782
-17.7105930571678 -17.767403081811782
-17.734871310549657 -17.767403081811782
-17.739153373370385 -17.767403081811782
-17.77208996271603 -17.767403081811782
iteration 0 311 0.004686880904248625
-17.707412905517987 -17.77208996271603
-17.76678581622074 -17.77208996271603
-17.74798880295797 -17.77208996271603
-17.780071208171364 -17.77208996271603
iteration 0 315 0.007981245455333408
-17.75263773409033 -17.780071208171364
-17.784399969835334 -17.780071208171364
iteration 0 317 0.0043287616639702264
-17.72541693523004 -17.784399969

-17.92407433108322 -17.980650491197373
-17.94389550497352 -17.980650491197373
-17.959354994542977 -17.980650491197373
-17.928108059814495 -17.980650491197373
-17.965640512996295 -17.980650491197373
-17.92647863460657 -17.980650491197373
-17.96098499523711 -17.980650491197373
-17.94106352886975 -17.980650491197373
-17.965404945462872 -17.980650491197373
-17.933366528869108 -17.980650491197373
-17.97450618125831 -17.980650491197373
-17.927888695408754 -17.980650491197373
-17.931393314416873 -17.980650491197373
-17.931642243569662 -17.980650491197373
-17.959827871958208 -17.980650491197373
-17.926841730946954 -17.980650491197373
-17.95861533294928 -17.980650491197373
-17.93411343586846 -17.980650491197373
-17.96879858164472 -17.980650491197373
-17.912909554583237 -17.980650491197373
-17.94657130145298 -17.980650491197373
-17.946854189376435 -17.980650491197373
-17.96496369729575 -17.980650491197373
-17.928544903659958 -17.980650491197373
-17.97070843605223 -17.980650491197373
-17.96282985

KeyboardInterrupt: 

In [ ]:
class Sim1():
    # Trick 1: Add rho||x||^2 /2 to the objectives 
    def __init__(self, n_train, deg, eps):
        self.n_train = n_train
        self.deg = deg
        self.eps = eps
        self.init = np.random.normal(0, 1, (d,p))
        self.B = np.zeros((d,p))
        self.B_star = np.random.binomial(size = (d,p), n = 1, p = 0.5)

        self.X_train = np.zeros((self.n_train,p))      
        for i in range(self.n_train):
            self.X_train[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_train = np.zeros((self.n_train,d))
        for i in range(self.n_train):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_train[i,j] = np.power((self.B_star @ self.X_train[i])[j] / np.sqrt(p) + 3, self.deg) * noise
        
        self.n_test = 1000

        self.X_test = np.zeros((self.n_test,p))
        for i in range(self.n_test):
            self.X_test[i] = np.random.multivariate_normal(np.zeros(p), np.eye(p))

        self.c_test = np.zeros((self.n_test,d))
        for i in range(self.n_test):
            for j in range(d):
                noise = np.random.uniform(1 - self.eps, 1 + self.eps)
                self.c_test[i,j] = np.power((self.B_star @ self.X_test[i])[j] / np.sqrt(p) + 3, self.deg) * noise

        self.dca_norm_spo_loss = 0
        self.sp_norm_spo_loss = 0 

    def subgrad_g(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(alpha * B @ X[i] - c[i]), X[i]) for i in range(n)], axis=0)
        return s / n + rho * B

    def subgrad_h(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([np.outer(-alpha * w_star(B @ X[i]), X[i]) for i in range(n)], axis=0)
        return s / n + rho * B

    def G(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] for i in range(n)])
        return s / n + (rho / 2) * np.sum(np.multiply(B,B))

    def H(self, B, X, c, alpha, rho):
        n = len(X)
        s = np.sum([alpha * supp(- B @ X[i])[0] for i in range(n)])
        return s / n + (rho / 2) * np.sum(np.multiply(B,B))

    def DC(self, B, X, c, alpha, rho):
        return self.G(B, X, c, alpha, rho) - self.H(B, X, c, alpha, rho)

    def subgrd_obj(self, B, C, X, c, alpha, rho):
        return self.G(B, X, c, alpha, rho) - np.sum(np.multiply(B, C))

    def fit_dca(self, alpha, gamma, rho):
        self.B = self.init

        T = 10000 # number of iterations
        tol = 1e-5 # tolerance

        for k in range(T):
            C = self.subgrad_h(self.B, self.X_train, self.c_train, alpha, rho)

            B_prime = self.B
            for l in range(T):
                B_prime_new = B_prime - (gamma/np.sqrt(l+1)) * (self.subgrad_g(B_prime, self.X_train, self.c_train, alpha, rho) - C)

                obj_gap = np.abs(self.subgrd_obj(B_prime_new, C, self.X_train, self.c_train, alpha, rho) \
                                 - self.subgrd_obj(B_prime, C, self.X_train, self.c_train, alpha, rho))
                print("iteration", k, l, obj_gap)
                if obj_gap < tol:
                    B_new = B_prime_new
                    break
                B_prime = B_prime_new
            
            dc_gap = np.abs(self.DC(B_new, self.X_train, self.c_train, alpha, rho) \
                      - self.DC(self.B, self.X_train, self.c_train, alpha, rho))
            if dc_gap < tol:
                break
            print("DC Gap:", dc_gap)
            
            self.B = B_new


        print("DCA Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i]) \
                                                          for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.dca_norm_spo_loss = numerator / denominator


    def subgrad_sp(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([np.outer(alpha * (w_star(c[i]) - w_star(alpha * B @ X[i] - c[i])), X[i]) for i in range(n)], axis=0)
        return s / n

    def spo_plus(self, B, X, c, alpha):
        n = len(X)
        s = np.sum([supp(c[i] - alpha * B @ X[i])[0] + alpha * np.dot(B @ X[i], w_star(c[i])) for i in range(n)])
        return s / n

    def fit_sp(self, alpha, gamma):
        self.B = self.init
    
        T = 10000 # number of iterations
        tol = 1e-5 # tolerance

        for k in range(T):
            B_new = self.B - (gamma/np.sqrt(k+1)) * self.subgrad_sp(self.B, self.X_train, self.c_train, alpha) 
            
            obj_gap = np.abs(self.spo_plus(B_new, self.X_train, self.c_train, alpha) \
                             - self.spo_plus(self.B, self.X_train, self.c_train, alpha))
            print("iteration", k, obj_gap)
            if obj_gap < tol:
                break
            self.B = B_new

        print("SPO+ Converged: Training SPO Loss", np.sum([spo_loss(self.c_train[i], B_new @ self.X_train[i])\
                                                           for i in range(self.n_train)]))

        numerator = np.sum([spo_loss(self.c_test[i], B_new @ self.X_test[i]) for i in range(self.n_test)])
        denominator = np.sum([np.dot(self.c_test[i], w_star(self.c_test[i])) for i in range(self.n_test)])

        self.sp_norm_spo_loss = numerator / denominator

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

deg = 1
eps = 0

num_trials = 5
DCA_loss = []
SP_loss = []

for trials in range(num_trials):
    sim = SimProx(n_train=100, deg=deg, eps=eps)
    sim.fit_dca(alpha = 2, gamma = 0.1)
    print("DCA Norm SPO Loss:", sim.dca_norm_spo_loss)
    DCA_loss.append(sim.dca_norm_spo_loss)
    sim.fit_sp(alpha = 2, gamma = 0.1)
    print("SPO+ Norm SPO Loss:", sim.sp_norm_spo_loss)
    SP_loss.append(sim.sp_norm_spo_loss)



# DATAFRAMES WITH TRIAL COLUMN ASSIGNED
df1 = pd.DataFrame(np.vstack((DCA_loss, SP_loss)).T, columns=["DCA", "SPO+"]).assign(deg=1)

cdf = pd.concat([df1])                                # CONCATENATE
mdf = pd.melt(cdf, id_vars=['deg'], var_name=['Loss'])      # MELT

ax = sns.boxplot(x="deg", y="value", hue="Loss", data=mdf)  # RUN PLOT   
plt.show()

plt.clf()
plt.close()

iteration 0 0 inf
iteration 0 1 0.6957195607345739
iteration 0 2 0.5580632174836531
iteration 0 3 0.4828322306784969
iteration 0 4 0.4344726366655074
iteration 0 5 0.39811170700000176
iteration 0 6 0.36782522328843115
iteration 0 7 0.3461276103654498
iteration 0 8 0.3265614738517222
iteration 0 9 0.30689940828379747
iteration 0 10 0.2940886788991044
iteration 0 11 0.28196647638205974
iteration 0 12 0.2695506915227952
iteration 0 13 0.25934172967579805
iteration 0 14 0.24919919491854436
iteration 0 15 0.23874986391400022
iteration 0 16 0.23285012537745509
iteration 0 17 0.22664561912161574
iteration 0 18 0.21997330667081272
iteration 0 19 0.2149150438596088
iteration 0 20 0.21002178706820374
iteration 0 21 0.20545511455027565
iteration 0 22 0.2011513238529936
iteration 0 23 0.1962290311737398
iteration 0 24 0.1919936846076986
iteration 0 25 0.1883983475518125
iteration 0 26 0.18493817575519245
iteration 0 27 0.1816155720847057
iteration 0 28 0.177528704952449
iteration 0 29 0.1741883661

KeyboardInterrupt: 